# Exp003: Search for constituents in Corpora
This is some instructive experimentation of how to find certain syntax in a corpus, e.g. Brown.

In [10]:
import torch
import nltk
import benepar

Download the Brown Corpus that contains parsed English sentences.

In [11]:
from nltk.corpus import brown
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /Users/dglandorf/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/dglandorf/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

Download the Berkeley Neural Parser

In [3]:
benepar.download('benepar_en3')
parser = benepar.Parser("benepar_en3")

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /Users/dglandorf/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
/Users/dglandorf/opt/miniconda3/envs/gctg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Define a function to parse sentences with the Berkeley Neural Parser

In [12]:
def parse_sentence(words, tags):
    """
    Parses a sentence using the given words and tags.

    Args:
        words (list): A list of words in the sentence.
        tags (list): A list of tags corresponding to each word.

    Returns:
        object: The parsed sentence object.

    """
    input_sentence = benepar.InputSentence(
        words=words,
        space_after=[True] * len(words),  # Assuming space after each word
        tags=tags,
        escaped_words=words
    )
    return parser.parse(input_sentence)

Define search patterns

In [13]:
def search_adj_list(tree):
    """
    Searches for a subtree in the given tree that represents an adjective phrase (ADJP)
    with at least two adjective (JJ) nodes and at least one comma (,) node.

    Args:
        tree (Tree): The tree to search in.

    Returns:
        bool: True if a matching subtree is found, False otherwise.
    """
    for subtree in tree.subtrees():
        if subtree.label() == 'ADJP':
            jj_count = 0
            comma_count = 0
            for node in subtree:
                if node.label() == 'JJ':
                    jj_count += 1
                elif node.label() == ',':
                    comma_count += 1

            if jj_count >= 2 and comma_count >= 1:
                return True
    return False

In [15]:
def find_adj_before_noun(tree, adjectives):
    """
    Finds adjectives that appear before a noun in a given tree.

    Parameters:
    - tree (nltk.tree.Tree): The tree to search for adjectives and nouns.
    - adjectives (list): A list of adjectives to search for.

    Returns:
    - matches (list): A list of tuples containing the adjective and the noun it appears before.
    """
    for subtree in tree.subtrees(lambda t: t.label() == 'NP'):  # Looking within Noun Phrases
        
        for i in range(len(subtree) - 1):  # Iterate over elements in the NP, stopping one short of the end
            current_node = subtree[i]
            next_node = subtree[i + 1]
            # Check if the current node is one of the specified adjectives and the next node is a noun
            if current_node.label() == 'JJ' and current_node[0] in adjectives \
               and next_node.label().startswith('NN'):
                return True

In [21]:
def find_perfect_tense(tree):
    """
    Finds sentences with a perfect tense.

    Args:
        tree (Tree): The parse tree of a sentence.

    Returns:
        bool: True if a sentence with a perfect tense is found, False otherwise.
    """
    for subtree in tree.subtrees():
        # search for 've/have/hast/hath/had + past participle
        # example tree string: (S (NP (PRP She)) (VP (VBZ has) (VP (VBN seen) (NP (PRP it)))) (. .))
        
        if subtree.label() == 'VP' and subtree[0].label=="VBN":
            print(subtree[1])
            print(subtree[1])
            if (subtree[0] == 'has' or subtree[0].label() == 'have') \
            and subtree[1][0].label == 'VBN':
                return True


Iterate over the corpus and apply the pattern.

In [22]:
for sent in brown.tagged_sents()[:100]:
    words, nltk_tags = zip(*sent)
    tree = parse_sentence(words, nltk_tags)
    #if search_adj_list(tree):
    #    print("Adjective list pattern in sentence:", ' '.join(tree.leaves()))
    #if find_adj_before_noun(tree, ['main', 'only']):
    #    print("Adj/noun pattern in sentence:", ' '.join(tree.leaves()))
    if find_perfect_tense(tree):
        print("Perfect tense pattern in sentence:", ' '.join(tree.leaves()))


/Users/dglandorf/opt/miniconda3/envs/gctg/lib/python3.10/site-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


(VP
  (VBD said)
  (NP (NR Friday))
  (SBAR
    (S
      (NP
        (NP (AT an) (NN investigation))
        (PP
          (IN of)
          (NP
            (NP (NP$ Atlanta's))
            (JJ recent)
            (NN primary)
            (NN election))))
      (VP
        (VBD produced)
        (`` ``)
        (NP
          (AT no)
          (NN evidence)
          ('' '')
          (SBAR
            (CS that)
            (S
              (NP (DTI any) (NNS irregularities))
              (VP (VBD took) (NP (NN place))))))))))
(VP
  (VBD produced)
  (`` ``)
  (NP
    (AT no)
    (NN evidence)
    ('' '')
    (SBAR
      (CS that)
      (S
        (NP (DTI any) (NNS irregularities))
        (VP (VBD took) (NP (NN place)))))))
(VP (VBD took) (NP (NN place)))
(VP
  (VBD said)
  (PP (IN in) (NP (NN term-end) (NNS presentments)))
  (SBAR
    (CS that)
    (S
      (NP
        (NP
          (AT the)
          (NN-TL City)
          (JJ-TL Executive)
          (NN-TL Committee))
        (, ,)

KeyboardInterrupt: 